Load libraries

In [67]:
## For some reason, when loading mlogit, the notebook can't find package 'statmod' so I specify it's location
library(statmod, lib.loc='D:\\Applications\\Anaconda2\\pkgs\\r-statmod-1.4.30-r3.4.1_0\\lib\\R\\library\\')
require(mlogit)
require(ggplot2)
require(reshape2)
require(lme4)
require(compiler)
require(parallel)
require(car)
require(boot)
require(dplyr)
require(sjstats)
require(broom)

# Load data and set factors

In [68]:
mydata <- read.csv("C:\\Users\\Sarah\\Documents\\Personal Content\\Lab_study_data\\all_massaged_data\\dataframe_all_factors_for_analysis.txt",sep = '\t')
# sid is the student number
mydata$sid <- factor(mydata$sid)
mydata$sim_index <- factor(mydata$sim_index)
mydata$lab_experience <- factor(mydata$lab_experience)
mydata$similar_sim <- factor(mydata$similar_sim)
mydata$cvs_graph <- factor(mydata$cvs_graph)
mydata$cvs_table <- factor(mydata$cvs_table)
mydata$quant_score <- factor(mydata$quant_score)
# mydata$main <- factor(mydata$main)
# mydata$pre <- factor(mydata$pre)

Here is what our data looks like:

In [69]:
head(mydata)
# colnames(mydata)

sid,sim,variable,pre,main,cvs_graph,cvs_table,cvs_table_only,qual_score,quant_score,...,pre_with_ident,main_with_ident,CVS_context,use_table,use_graph,use_concentration,use_width,use_area,use_separation,use_all_vars
10127163,L,Concentration,0,2,1,1,0,1,1,...,1,3,2,1,1,1,1,1,1,4
10127163,L,Width,0,2,1,1,0,1,1,...,1,3,2,1,1,1,1,1,1,4
10127163,C,Area,2,2,1,1,0,1,1,...,3,3,2,1,1,1,1,1,1,4
10127163,C,Separation,2,2,1,1,0,1,1,...,3,3,2,1,1,1,1,1,1,4
10232160,L,Concentration,0,0,1,1,0,1,1,...,1,1,2,1,1,1,1,1,1,4
10232160,L,Width,0,0,0,0,0,1,1,...,1,1,0,1,1,1,1,1,1,4


We have the following factors that change per variable:
* main (0,1,2), treated as a continuous variable
* pre (0,1,2), treated as a continuous variable
* quant_score (0 or 1)
* CVS_graph (0 or 1)
* CVS_table (0 or 1)

We have the following independant factors:
* sim_index (1 or 2, wither it was student's 1st or 2nd activity)
* variable (thus don't include sim as a variable)
* student attibutes:
   * lab_experience (0 or 1 if students have prior undergraduate physics or chemistry lab experience)
   * similar_sim (0 or 1 if they have used a similar simulation)
   * prior_number_virtual_labs (levels from 0 to 3 depending on the number of virtual labs they have done in the past)

We ignore attitude components.

For main and pre score:
* score = 2 if they describe the correct relationship, ie. a correct quantitative model
* score = 1 if they describe the correct direction of the relationship, ie. they have a correct qual model but incorrect quant model OR if their quant model is incorrect but qualitatively correct
* score = 0 otherwise (i.e. all incorrect or only identified)

# Stat model 1: Prediction main model score as a continuous variable

Some resources:
* On SS Types: https://mcfromnz.wordpress.com/2011/03/02/anova-type-iiiiii-ss-explained/
* on drop() function to do type 3: https://www.statmethods.net/stats/anova.html
* On repeated measures: http://psych.wisc.edu/moore/Rpdf/610-R8_OneWayWithin.pdf, https://datascienceplus.com/two-way-anova-with-repeated-measures/
* the car package: https://cran.r-project.org/web/packages/car/car.pdf

## Complete model with interactions

Our model (without student factors) is:

    main  ~  cvs_table*variable + cvs_graph*variable
             + cvs_table*pre + cvs_graph*pre
             + sim_index + sid
             
We run a type III Anova:

In [70]:
lm1 = lm(main
        ~  cvs_table*variable + cvs_graph*variable + cvs_table*pre + cvs_graph*pre + sim_index + sid,
         data=mydata)
results1 = Anova(lm1, type=3)
results_table1 = tidy(results1)
results_table1$eta <- results_table1$sumsq/(results_table1$sumsq + results_table1$sumsq[dim(results_table1)[1]])
results_table1

term,sumsq,df,statistic,p.value,eta
(Intercept),7.805831e+00,1,34.747383469,7.642655e-09,7.541526e-02
cvs_table,2.646537e-02,1,0.117809663,7.315917e-01,2.764720e-04
variable,2.962562e-01,3,0.439591352,7.247968e-01,3.086160e-03
cvs_graph,1.019934e+00,1,4.540199846,3.367984e-02,1.054536e-02
pre,9.773337e-04,1,0.004350567,9.474415e-01,1.021249e-05
sim_index,2.740467e+00,1,12.199093390,5.281336e-04,2.783916e-02
sid,9.881870e+01,146,3.012931588,1.176573e-18,5.080195e-01
cvs_table:variable,1.212752e+00,3,1.799508506,1.465681e-01,1.251401e-02
variable:cvs_graph,4.663895e-02,3,0.069203884,9.763197e-01,4.871139e-04
cvs_table:pre,1.981195e-03,1,0.008819222,9.252242e-01,2.070197e-05


None of the interactions are significant so let's move to a simpler model.

## Simple model without interaction

Our model (without student factors) is:

    main  ~  cvs_table + cvs_graph + variable
             + pre + sim_index + sid
             
We run a type II Anova:

In [71]:
lm1 = lm(main
        ~  cvs_table + cvs_graph + variable + pre + sim_index + sid,
         data=mydata)
results1 = Anova(lm1, type=2)
results_table1 = tidy(results1)
results_table1$eta <- results_table1$sumsq/(results_table1$sumsq + results_table1$sumsq[dim(results_table1)[1]])
results_table1

term,sumsq,df,statistic,p.value,eta
cvs_table,2.875905e-03,1,0.01265297,9.104905e-01,2.915345e-05
cvs_graph,2.846821e+00,1,12.52501086,4.449018e-04,2.804996e-02
variable,2.341790e+00,3,3.43434961,1.698431e-02,2.318924e-02
pre,5.063759e-01,1,2.22787537,1.362667e-01,5.107137e-03
sim_index,2.769224e+00,1,12.18360845,5.314560e-04,2.730627e-02
sid,1.003316e+02,146,3.02345002,6.702068e-19,5.042401e-01
Residuals,9.864427e+01,434,NA,NA,5.000000e-01


We see that, in order of significance and eta^2: cvs_graph, sim_index, and variable matter.

## Model by variable

### For Width

In [72]:
test <- subset(mydata, variable == "Width")
lm1 = lm(main
        ~  cvs_table + cvs_graph + pre + sim_index,
         data=test)
results1 = Anova(lm1, type=2)
results_table1 = tidy(results1)
results_table1$eta <- results_table1$sumsq/(results_table1$sumsq + results_table1$sumsq[dim(results_table1)[1]])
results_table1

term,sumsq,df,statistic,p.value,eta
cvs_table,2.0083545,1,5.6774156,0.01850865,0.038444711
cvs_graph,1.2765488,1,3.6086747,0.05950658,0.024783377
pre,0.2706068,1,0.7649782,0.38325199,0.005358304
sim_index,1.5010825,1,4.2434090,0.04122893,0.029016070
Residuals,50.2317177,142,NA,NA,0.500000000


### For Area

In [73]:
test <- subset(mydata, variable == "Area")
lm1 = lm(main
        ~  cvs_table + cvs_graph + pre + sim_index,
         data=test)
results1 = Anova(lm1, type=2)
results_table1 = tidy(results1)
results_table1$eta <- results_table1$sumsq/(results_table1$sumsq + results_table1$sumsq[dim(results_table1)[1]])
results_table1

term,sumsq,df,statistic,p.value,eta
cvs_table,0.306401330,1,0.99824432,0.319435228,0.0069808152
cvs_graph,1.745938586,1,5.68820401,0.018399673,0.0385149515
pre,2.643065816,1,8.61101169,0.003897894,0.0571738520
sim_index,0.009813715,1,0.03197273,0.858342398,0.0002251094
Residuals,43.585511150,142,NA,NA,0.5000000000


### For Concentration

In [74]:
test <- subset(mydata, variable == "Concentration")
lm1 = lm(main
        ~  cvs_table + cvs_graph + pre + sim_index,
         data=test)
results1 = Anova(lm1, type=2)
results_table1 = tidy(results1)
results_table1$eta <- results_table1$sumsq/(results_table1$sumsq + results_table1$sumsq[dim(results_table1)[1]])
results_table1

term,sumsq,df,statistic,p.value,eta
cvs_table,0.2787357,1,0.8475172,0.358817000,0.00593302
cvs_graph,2.5602716,1,7.7847011,0.005993397,0.05197261
pre,1.7380768,1,5.2847552,0.022972198,0.03588121
sim_index,2.2443126,1,6.8240039,0.009961487,0.04585284
Residuals,46.7016722,142,NA,NA,0.50000000


### For Separation

In [75]:
test <- subset(mydata, variable == "Separation")
lm1 = lm(main
        ~  cvs_table + cvs_graph + pre + sim_index,
         data=test)
results1 = Anova(lm1, type=2)
results_table1 = tidy(results1)
results_table1$eta <- results_table1$sumsq/(results_table1$sumsq + results_table1$sumsq[dim(results_table1)[1]])
results_table1

term,sumsq,df,statistic,p.value,eta
cvs_table,0.4788152,1,1.302363,0.25570297,0.009088217
cvs_graph,1.6857681,1,4.585238,0.03395547,0.031280356
pre,2.5078567,1,6.821295,0.00997590,0.045835475
sim_index,0.3905307,1,1.062232,0.30445896,0.007424964
Residuals,52.2064594,142,NA,NA,0.500000000


In [93]:
colMeans(test["main"])

main 
1.217687

# Stat model 2: Predicting transfer data

## Excluding student main worksheet score

### Complete model with interactions

Our model is:

    quant_score  ~  cvs_table*variable + cvs_graph*variable
             + cvs_table*pre + cvs_graph*pre
             + sim_index + sid
             
We run a logistic regression:

In [77]:
mixed1 <- glmer(
    quant_score
    ~ cvs_table*variable + cvs_graph*variable + cvs_table*pre + cvs_graph*pre + sim_index + (1 | sid),
           data = mydata, family = binomial, 
           control = glmerControl(optimizer = "bobyqa"), nAGQ = 10)
summary(mixed1)


Correlation matrix not shown by default, as p = 16 > 12.
Use print(obj, correlation=TRUE)  or
	 vcov(obj)	 if you need it



Generalized linear mixed model fit by maximum likelihood (Adaptive
  Gauss-Hermite Quadrature, nAGQ = 10) [glmerMod]
 Family: binomial  ( logit )
Formula: 
quant_score ~ cvs_table * variable + cvs_graph * variable + cvs_table *  
    pre + cvs_graph * pre + sim_index + (1 | sid)
   Data: mydata
Control: glmerControl(optimizer = "bobyqa")

     AIC      BIC   logLik deviance df.resid 
   670.0    744.4   -318.0    636.0      571 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-3.2052 -0.4659  0.2881  0.4786  1.2799 

Random effects:
 Groups Name        Variance Std.Dev.
 sid    (Intercept) 3.735    1.932   
Number of obs: 588, groups:  sid, 147

Fixed effects:
                                 Estimate Std. Error z value Pr(>|z|)  
(Intercept)                       1.16394    0.46704   2.492   0.0127 *
cvs_table1                        0.28970    0.78194   0.370   0.7110  
variableConcentration            -0.19343    0.55513  -0.348   0.7275  
variableSeparation            

**(non log) Odds ratio with confidence intervals**

In [78]:
cc <- confint(mixed1,parm="beta_",method="Wald")
ctab <- cbind(est=fixef(mixed1),cc)
rtab <- exp(ctab)
print(rtab,digits=3)

                                   est  2.5 % 97.5 %
(Intercept)                      3.203 1.2822   8.00
cvs_table1                       1.336 0.2886   6.19
variableConcentration            0.824 0.2776   2.45
variableSeparation               0.346 0.1259   0.95
variableWidth                    1.597 0.5387   4.73
cvs_graph1                       0.503 0.1060   2.39
pre                              2.657 1.1025   6.40
sim_index2                       1.300 0.8226   2.05
cvs_table1:variableConcentration 0.678 0.0865   5.32
cvs_table1:variableSeparation    0.971 0.1345   7.01
cvs_table1:variableWidth         0.474 0.0591   3.80
variableConcentration:cvs_graph1 2.760 0.3685  20.68
variableSeparation:cvs_graph1    1.772 0.2571  12.21
variableWidth:cvs_graph1         1.205 0.1635   8.88
cvs_table1:pre                   0.460 0.1186   1.79
cvs_graph1:pre                   1.572 0.4528   5.46


Again interactions are not significant, so we stick to a simpler model.

### Simple model without interactions

Our model is:

    quant_score  ~  cvs_table + cvs_graph + variable
                     + pre + sim_index + sid
             
We run a logistic regression:

In [79]:
mixed1 <- glmer(
    quant_score
    ~ cvs_table + cvs_graph + variable + sim_index + pre + (1 | sid),
           data = mydata, family = binomial, 
           control = glmerControl(optimizer = "bobyqa"), nAGQ = 10)
summary(mixed1)

Generalized linear mixed model fit by maximum likelihood (Adaptive
  Gauss-Hermite Quadrature, nAGQ = 10) [glmerMod]
 Family: binomial  ( logit )
Formula: quant_score ~ cvs_table + cvs_graph + variable + sim_index +  
    pre + (1 | sid)
   Data: mydata
Control: glmerControl(optimizer = "bobyqa")

     AIC      BIC   logLik deviance df.resid 
   658.8    698.2   -320.4    640.8      579 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-2.6604 -0.4832  0.2932  0.4892  1.3620 

Random effects:
 Groups Name        Variance Std.Dev.
 sid    (Intercept) 3.461    1.86    
Number of obs: 588, groups:  sid, 147

Fixed effects:
                      Estimate Std. Error z value Pr(>|z|)    
(Intercept)            1.21488    0.36514   3.327 0.000877 ***
cvs_table1            -0.23617    0.41210  -0.573 0.566580    
cvs_graph1            -0.05100    0.42562  -0.120 0.904629    
variableConcentration  0.03205    0.32643   0.098 0.921793    
variableSeparation    -0.81255    0.31677  -2

**(non log) Odds ratio with confidence intervals**

In [80]:
cc <- confint(mixed1,parm="beta_",method="Wald")
ctab <- cbind(est=fixef(mixed1),cc)
rtab <- exp(ctab)
print(rtab,digits=3)

                        est 2.5 % 97.5 %
(Intercept)           3.370 1.647  6.893
cvs_table1            0.790 0.352  1.771
cvs_graph1            0.950 0.413  2.188
variableConcentration 1.033 0.545  1.958
variableSeparation    0.444 0.238  0.826
variableWidth         1.072 0.567  2.024
sim_index2            1.275 0.813  2.001
pre                   1.831 1.158  2.894


As expected, CVS doesn't predict quant transfer scores, only variable does.

## Including student main worksheet score
as a continuous variable

### Complete model with interactions

Our model is:

    quant_score  ~  main + cvs_table*variable + cvs_graph*variable
                    + cvs_table*pre + cvs_graph*pre
                    + sim_index + sid
             
We run a logistic regression:

In [81]:
mixed1 <- glmer(
    quant_score
    ~ main + cvs_table*variable + cvs_graph*variable + cvs_table*pre + cvs_graph*pre + sim_index + (1 | sid),
           data = mydata, family = binomial, 
           control = glmerControl(optimizer = "bobyqa"), nAGQ = 10)
summary(mixed1)


Correlation matrix not shown by default, as p = 17 > 12.
Use print(obj, correlation=TRUE)  or
	 vcov(obj)	 if you need it



Generalized linear mixed model fit by maximum likelihood (Adaptive
  Gauss-Hermite Quadrature, nAGQ = 10) [glmerMod]
 Family: binomial  ( logit )
Formula: quant_score ~ main + cvs_table * variable + cvs_graph * variable +  
    cvs_table * pre + cvs_graph * pre + sim_index + (1 | sid)
   Data: mydata
Control: glmerControl(optimizer = "bobyqa")

     AIC      BIC   logLik deviance df.resid 
   639.2    718.0   -301.6    603.2      570 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-2.8868 -0.3810  0.2352  0.4439  2.6622 

Random effects:
 Groups Name        Variance Std.Dev.
 sid    (Intercept) 4.377    2.092   
Number of obs: 588, groups:  sid, 147

Fixed effects:
                                 Estimate Std. Error z value Pr(>|z|)    
(Intercept)                      -0.22567    0.55203  -0.409   0.6827    
main                              1.46100    0.28806   5.072 3.94e-07 ***
cvs_table1                        0.21502    0.82361   0.261   0.7940    
variableConcentr

**(non log) Odds ratio with confidence intervals**

In [82]:
cc <- confint(mixed1,parm="beta_",method="Wald")
ctab <- cbind(est=fixef(mixed1),cc)
rtab <- exp(ctab)
print(rtab,digits=3)

                                   est  2.5 % 97.5 %
(Intercept)                      0.798 0.2705  2.354
main                             4.310 2.4508  7.581
cvs_table1                       1.240 0.2468  6.229
variableConcentration            0.733 0.2350  2.287
variableSeparation               0.285 0.0987  0.823
variableWidth                    1.684 0.5346  5.303
cvs_graph1                       0.276 0.0516  1.478
pre                              2.581 1.0100  6.594
sim_index2                       1.098 0.6741  1.789
cvs_table1:variableConcentration 0.653 0.0747  5.706
cvs_table1:variableSeparation    1.569 0.1983 12.420
cvs_table1:variableWidth         0.352 0.0394  3.136
variableConcentration:cvs_graph1 3.150 0.3724 26.636
variableSeparation:cvs_graph1    1.893 0.2504 14.313
variableWidth:cvs_graph1         1.417 0.1742 11.524
cvs_table1:pre                   0.445 0.1004  1.971
cvs_graph1:pre                   1.467 0.3715  5.794


Again interactions are not significant, so we stick to a simpler model.

### Simple model without interactions

Our model is:

    quant_score  ~  main + cvs_table + cvs_graph + variable
                     + pre + sim_index + sid
             
We run a logistic regression:

In [83]:
mixed1 <- glmer(
    quant_score
    ~ main + cvs_table + cvs_graph + variable + sim_index + pre + (1 | sid),
           data = mydata, family = binomial, 
           control = glmerControl(optimizer = "bobyqa"), nAGQ = 10)
summary(mixed1)

Generalized linear mixed model fit by maximum likelihood (Adaptive
  Gauss-Hermite Quadrature, nAGQ = 10) [glmerMod]
 Family: binomial  ( logit )
Formula: quant_score ~ main + cvs_table + cvs_graph + variable + sim_index +  
    pre + (1 | sid)
   Data: mydata
Control: glmerControl(optimizer = "bobyqa")

     AIC      BIC   logLik deviance df.resid 
   630.9    674.7   -305.5    610.9      578 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-2.8731 -0.4163  0.2662  0.4557  2.0194 

Random effects:
 Groups Name        Variance Std.Dev.
 sid    (Intercept) 3.965    1.991   
Number of obs: 588, groups:  sid, 147

Fixed effects:
                       Estimate Std. Error z value Pr(>|z|)    
(Intercept)           -0.107637   0.452839  -0.238   0.8121    
main                   1.340418   0.271828   4.931 8.18e-07 ***
cvs_table1            -0.269440   0.439074  -0.614   0.5394    
cvs_graph1            -0.539544   0.468049  -1.153   0.2490    
variableConcentration -0.046691  

**(non log) Odds ratio with confidence intervals**

In [84]:
cc <- confint(mixed1,parm="beta_",method="Wald")
ctab <- cbind(est=fixef(mixed1),cc)
rtab <- exp(ctab)
print(rtab,digits=3)

                        est 2.5 % 97.5 %
(Intercept)           0.898 0.370  2.181
main                  3.821 2.243  6.509
cvs_table1            0.764 0.323  1.806
cvs_graph1            0.583 0.233  1.459
variableConcentration 0.954 0.488  1.865
variableSeparation    0.489 0.256  0.937
variableWidth         1.009 0.518  1.967
sim_index2            1.097 0.681  1.765
pre                   1.668 1.020  2.728


## Discussion on all 4 models (w. w/o interaction and w. w/o main)
What we notice:
* cvs_graph never matters
* main matters
* pre matters
* variable matters
* sim_index doesn't matter...

# Stat model 3: Predicting the use of CVS graph

Our model is:

    cvs_graph  ~ variable + pre + sim_index + sid
                 + lab_experience + similar_sim + prior_number_virtual_labs
             
We run a logistic regression:

In [85]:
mixed <- glmer(
    cvs_graph
    ~ variable + sim_index + pre
    + lab_experience + similar_sim + prior_number_virtual_labs + (1 | sid),
           data = mydata, family = binomial, 
           control = glmerControl(optimizer = "bobyqa"), nAGQ = 10)

summary(mixed)

Generalized linear mixed model fit by maximum likelihood (Adaptive
  Gauss-Hermite Quadrature, nAGQ = 10) [glmerMod]
 Family: binomial  ( logit )
Formula: 
cvs_graph ~ variable + sim_index + pre + lab_experience + similar_sim +  
    prior_number_virtual_labs + (1 | sid)
   Data: mydata
Control: glmerControl(optimizer = "bobyqa")

     AIC      BIC   logLik deviance df.resid 
   603.7    647.5   -291.9    583.7      578 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-2.0406 -0.3283 -0.1003  0.3351  2.8259 

Random effects:
 Groups Name        Variance Std.Dev.
 sid    (Intercept) 10.79    3.284   
Number of obs: 588, groups:  sid, 147

Fixed effects:
                          Estimate Std. Error z value Pr(>|z|)    
(Intercept)                -5.6625     1.3361  -4.238 2.25e-05 ***
variableConcentration       0.7366     0.4057   1.816  0.06943 .  
variableSeparation         -0.1014     0.3731  -0.272  0.78572    
variableWidth               0.2744     0.3993   0.687  0.4

**(non log) Odds ratio with confidence intervals**

In [86]:
cc <- confint(mixed1,parm="beta_",method="Wald")
ctab <- cbind(est=fixef(mixed1),cc)
rtab <- exp(ctab)
print(rtab,digits=3)

                        est 2.5 % 97.5 %
(Intercept)           0.898 0.370  2.181
main                  3.821 2.243  6.509
cvs_table1            0.764 0.323  1.806
cvs_graph1            0.583 0.233  1.459
variableConcentration 0.954 0.488  1.865
variableSeparation    0.489 0.256  0.937
variableWidth         1.009 0.518  1.967
sim_index2            1.097 0.681  1.765
pre                   1.668 1.020  2.728









____________________________________________________________________________





# OTHER VERSION OF ANALYSES - keep for historical purposes
Even though we decided not to include them or do analyses this way, we keep the code to run them here just in case.

First we reload the data, in case some factors have changed from continuous to categorical variables

In [87]:
# mydata <- read.csv("C:\\Users\\Sarah\\Documents\\Personal Content\\Lab_study_data\\all_massaged_data\\dataframe_all_factors_for_analysis.txt",sep = '\t')
# # sid is the student number
# mydata$sid <- factor(mydata$sid)
# mydata$sim_index <- factor(mydata$sim_index)
# mydata$lab_experience <- factor(mydata$lab_experience)
# mydata$similar_sim <- factor(mydata$similar_sim)
# mydata$cvs_graph <- factor(mydata$cvs_graph)
# mydata$cvs_table <- factor(mydata$cvs_table)
# # mydata$main <- factor(mydata$main)
# # mydata$pre <- factor(mydata$pre)

## Stat model 1: Predicting main model scores as a categorical variable

First we transform the data in an extra wide format for the mlogit function.
Now every student has a row for each variable times type of model (0,1,2).
The "alt" is the model type (0,1,2) and "main" is True if that was the model type they got correct (and the others are always False for that variable).

In [88]:
# mydata$main <- factor(mydata$main)
# mydata$pre <- factor(mydata$pre)

In [89]:
# wide_mydata <- mlogit.data(mydata, shape = 'wide', choice = "main", id.var = "sid")
# head(wide_mydata, 5)

Then we run the mlogit model.

See the following: https://cran.r-project.org/web/packages/mlogit/vignettes/mlogit.pdf

Specifically, mixed in this document DOESN't mean with repeated measures. The "1 | " in the formula below tells it that some of the variables are individual specific.
The examples using the "Train" dataset is what I followed. See pages 3-7 for how to structure data and 22,23 for example of running mlogit.

In [90]:
# ml.mydata <- mlogit(main
#     ~ 1 | cvs_table + cvs_graph + variable + sim_index + pre
#     + lab_experience + similar_sim + prior_number_virtual_labs, wide_mydata)
# summary(ml.mydata)